In [35]:
import hdbscan
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd
from scipy.io import loadmat
#pyqtgraph -> fast plotting
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui
%gui qt5
import time

## Important parameters
<ul>
    <li> 
        min_cluster_size - smallest size grouping that you wish to consider a cluster
    </li>
    <li> 
        min_samples - a measure of how conservative you want you clustering to be. The larger 
        the value of min_samples you provide, the more conservative the clustering – more points will be 
        declared as noise, and clusters will be restricted to progressively more dense areas.
    </li>
</ul>

In [4]:
#initialise variables
lostSync = False

#valid header variables and constant
magicBytes = np.array([2,1,4,3,6,5,8,7], dtype= 'uint8')

isMagicOk = False
isDataOk = False
gotHeader = False

frameHeaderLength = 52 #52 bytes long
tlvHeaderLengthInBytes = 8
pointLengthInBytes = 16
frameNumber = 1
targetFrameNumber = 0
targetLengthInBytes = 68

In [26]:
#one person slow walk
#TLV Data Load
tlvData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\Matlab Data\\slowWalk.mat'))['tlvStream'][0]

In [11]:
#extract people walking 
walking = tlvData[200:500]
tlvData = walking

In [ ]:
#two people slow walk
#TLV Data Load
tlvData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\Matlab Data\\2PeopleMoving.mat'))['tlvStream'][0]

In [ ]:
#three people walk
#TLV Data Load
tlvData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\Matlab Data\\3PeopleWalking.mat'))['tlvStream'][0]

In [18]:
app = QtGui.QApplication([])
pg.setConfigOption('background','w')

In [20]:
win = pg.GraphicsWindow(title="")
plot1 = win.addPlot()
plot1.setXRange(-6,6)
plot1.setYRange(0,6)
plot1.setLabel('left',text = 'Y position (m)')
plot1.setLabel('bottom', text= 'X position (m)')
s1 = plot1.plot([],[],pen=None,symbol='o')

plot2 = win.addPlot()
plot2.setXRange(-6,6)
plot2.setYRange(0,6)
plot2.setLabel('left',text = 'Y position (m)')
plot2.setLabel('bottom', text= 'X position (m)')
s2 = plot2.plot([],[],pen=None,symbol='o')

In [40]:
centroidData = list();
for tlvStream in tlvData:
    tlvStream = np.frombuffer(tlvStream, dtype = 'uint8')
    
    #tlv header
    index = 0
    #tlv header parsing
    tlvType = tlvStream[index:index+4].view(dtype=np.uint32)
    tlvLength = tlvStream[index+4:index+8].view(dtype=np.uint32)
    
    index += tlvHeaderLengthInBytes
    tlvDataLength = tlvLength - tlvHeaderLengthInBytes

    if tlvType == 6: 
        numberOfPoints = tlvDataLength/pointLengthInBytes
        p = tlvStream[index:index+tlvDataLength[0]].view(np.single)
        pointCloud = np.reshape(p,(4, int(numberOfPoints)),order="F")

        if not(pointCloud is None):
            #constrain point cloud to within the effective sensor range
            #range 1 < x < 6
            #azimuth -50 deg to 50 deg
            #check whether corresponding range and azimuth data are within the constraints

            effectivePointCloud = np.array([])
            for index in range(0, len(pointCloud[0,:])):
                if (pointCloud[0,index] > 1 and pointCloud[0,index] < 6) \
                and (pointCloud[1, index] > -50*np.pi/180 \
                     and pointCloud[1, index] < 50*np.pi/180):
        
                    #concatenate columns to the new point cloud
                    if len(effectivePointCloud) == 0:
                        effectivePointCloud = np.reshape(pointCloud[:, index], (4,1), order="F")
                    else:
                        point = np.reshape(pointCloud[:, index], (4,1),order="F")
                        effectivePointCloud = np.hstack((effectivePointCloud, point))
                        

            if len(effectivePointCloud) != 0:
                posX = np.multiply(effectivePointCloud[0,:], np.sin(effectivePointCloud[1,:]))
                posY = np.multiply(effectivePointCloud[0,:], np.cos(effectivePointCloud[1,:]))
                positions = pd.DataFrame({'X':posX, 'Y':posY})
                if len(positions.values) > 5:
                    clusterer = hdbscan.HDBSCAN()
                    clusterer.fit(positions.values)

                    #extract clusters
                    s1.setData(positions.values[])
                    QtGui.QApplication.processEvents() 
                    time.sleep(0.04)

ValueError: Must provide either V or VI for Mahalanobis distance

In [7]:
#stores the cluster information
#cluster number for each data sample
#note that -1 is noise
clusterer.labels_

array([1, 0, 0, ..., 0, 1, 2], dtype=int64)

In [9]:
#number of clusters
clusterer.labels_.max() + 1

3

In [10]:
#hdbscan implements soft clustering - where each data point is assigned a cluster membership score ranging 
# from 0.0 to 1.0. A score of 0.0 represents a sample that is not in the cluster at all 
# (all noise points will get this score) while a score of 1.0 represents a sample that is 
# at the heart of the cluster (note that this is not the spatial centroid notion of core).

clusterer.probabilities_

#probabiity for each data point

array([0.63928435, 0.54577043, 0.68495415, ..., 0.75363376, 0.84837895,
       1.        ])

In [ ]:
#hbscan allows us to data associate using predict